# Analysis particles in NWES as function of Rep
In this notebook we analyse the trajectories of slow manifold Maxey-Riley-Gatinol (SM-MRG) particles in the NWES as   
function of the particle Reynods number $Re_p$. There are 2 types of SM-MRG particles: 
1. particles with a constant reynolds number (preset and constant for all particles in simulations).  
2. particles with aflexible reynolds number (reynolds number that is calculated at every simulation timestep based on the slip velocity).  
We compare SM-MRG particles with tracer (surface flow following) particles.


In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import Divider, Size
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta
from importlib import reload
import hexbin_functions as hexfunc
import h3
from shapely.geometry import shape, Point

hexfunc = reload(hexfunc)
# test version, it should be version 4
print(h3.__version__)

from analysis_functions_xr import trajectory_length, calc_tidal_av, skill_score, normalized_separation_dist
from analysis_functions import make_PDF, Haversine, make_lognormal_PDF
from particle_characteristics_functions import factor_drag_white1991, dynamic_viscosity_Sharqawy, stokes_relaxation_time, buoyancy_drifter

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
# markers and colors
Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.8, 3))#Magma(np.linspace(0.2, 0.9, 5))

color_array = np.array(['c','orange','purple','green','navy'])
marker_array = np.array(['s','o','>','p'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['-s','--o','-.>',':p'])

In [ ]:
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.25 #m
diameter_in =0.2
heigth_drifter = 0.041  # m (heigth drifter)
m_drifter = 0.905 # kg (mass drifter) 
rho_water = 1027 # kg/m3 
av_temp_NWES = 11.983276 # [degrees C]
av_salinity_NWES = 34.70449/1000 # [kg/kg]
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(T = av_temp_NWES, S = av_salinity_NWES) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
B = buoyancy_drifter(diameter  = diameter_in, heigth = heigth_drifter, mass = m_drifter, density_fluid = rho_water)
tau = stokes_relaxation_time(diameter = diameter, kinematic_viscosity = kinematic_viscosity_water, buoyancy = B)



In [ ]:
# import simulation data
# settings
Replist = np.array([100,450,1000]) # list of fixed input particle reynolds numbers used
coriolis = True 
gradient = True
# round numbers
B = round(B,2)
tau = int(tau)
runtime =  timedelta(days=30)# timedelta(days=10)
land_handling = 'anti_beaching'
nparticles = 52511 # number of particles
chunck_time = 100 # chuncking used in time
loc = 'NWES' # release location
Tmax=720 # number of timesteps



starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0), 
                       datetime(2023, 10, 1, 0, 0, 0, 0), 
                       datetime(2023, 11, 1, 0, 0, 0, 0) ,
                       datetime(2023, 12, 1, 0, 0, 0, 0) ,
                       datetime(2024, 1, 1, 0, 0, 0, 0),
                       datetime(2024, 2, 1, 0, 0, 0, 0)])

date_new = datetime(2023, 9, 1, 0, 0, 0, 0)

base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')

particle_types = ['tracer','inertial_SM_drag_Rep','inertial_SM_Rep_constant']
simtype = {'tracer':'tracer','tracer_random':'displaced tracer','inertial_SM_Rep_constant':'SM MR','inertial_Rep_constant':'full MR', 'inertial_SM_drag_Rep': 'flexible Re$_p$'}

basefiles = {'tracer':basefile_tracer,
             'inertial_Rep_constant':basefile_Rep_constant,
             'inertial_SM_Rep_constant':basefile_Rep_constant,
            'tracer_random':basefile_tracer_random,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag
             }

Repvalues = {'tracer':[None],
             'inertial_SM_Rep_constant':Replist,
             'inertial_Rep_constant':Replist,
             'inertial_SM_drag_Rep':[None]}


In [ ]:
# read in data
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}

        for Rep in Repvalues[pt]:
            data[pt][coriolis][Rep]={}

for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            
            for Rep in Repvalues[pt]:
                file = basefiles[pt].format(loc=loc,
                                            y_s=starttime.year,
                                            m_s=starttime.month,
                                            d_s=starttime.day,
                                            y_e=endtime.year,
                                            m_e=endtime.month,
                                            d_e=endtime.day,
                                            B = int(B * 1000), 
                                            tau = int(tau ),
                                            land_handling = land_handling, 
                                            coriolis = coriolis,
                                            gradient = gradient,
                                            particle_type = pt,
                                            Rep = Rep)
                ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) 

                data[pt][coriolis][Rep][date]= ds 

In [ ]:
# plot selection of trajectories
id =25

fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
colorlist=['k','cornflowerblue','blue']
markers = ['-','--','-.']
year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend= []

ax.coastlines()
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
for pt, color, marker in zip(particle_types,colorlist,markers):

        
        if(pt == 'inertial_SM_Rep_constant'):
                for Rep, color2 in zip(Replist,Magmalist):
                        ax.plot(data[pt][coriolis][Rep][date].lon[id,0::1].T,
                                data[pt][coriolis][Rep][date].lat[id,0::1].T,
                                marker,
                                color=color2);

                        legend.append('Re$_p^{\\mathrm{In}}$'+'={:d}'.format(Rep)) #simtype[pt]+
        else:
                ax.plot(data[pt][coriolis][None][date].lon[id,0::1].T,
                data[pt][coriolis][None][date].lat[id,0::1].T,
                marker,
                color=color,zorder=20);
                legend.append(simtype[pt])
ax.legend(legend,fontsize=18, handlelength = 1.5)#,loc=(1,0.))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}

ax.set_xlim(-14.6,-11.2)
ax.set_ylim(54.4,56)

ax.plot(data[pt][coriolis][1000][date].lon[id,0].T,
data[pt][coriolis][1000][date].lat[id,0].T,
'>',
color='darkgreen',zorder=20,markersize=15);
ax.set_title('hourly velocity fields',fontsize=18)
# fig.savefig('../figures/hourly_vs_daily/trajectory_hourly_Rep.pdf')



In [ ]:
# add tidal average lons and lats 
window=25
startnan=np.full((nparticles,25),np.nan)
endnan=np.full((nparticles,80),np.nan)
Tend=720
coriolis=True
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'

    for pt in particle_types:
        for Rep in Repvalues[pt]:
            da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lon,window)
            da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lat,window)

            data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lon_tidal_av = da_lon_tidal_av)
            data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lat_tidal_av = da_lat_tidal_av)
                 

## Trajectory Length

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        for Rep in Repvalues[pt]:
            ds =data[pt][coriolis][Rep][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            ds = ds.assign(traj_length = traj_length)
            traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
            data[pt][coriolis][Rep][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
    

            


In [ ]:
#  plot PDF of total trajectory length
fig,ax=plt.subplots()

Tmax = 720

legend=[]

for pt, color, marker in zip(particle_types,colorlist,markers):
    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist): 
            legend.append(' Re$_p$ ='+' {:d}'.format(Rep))
            data_arrays = list(data[pt][coriolis][Rep].values())
            combined_data = xr.concat(data_arrays, dim="starting_days")
            average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)
            ax.plot(bins,pdf,marker,color=color2)
    else:
        legend.append(simtype[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")
        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)
        ax.plot(bins,pdf,marker,color=color,zorder=20)

ax.legend(legend)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length (km)")


In [ ]:
# plot PDF difference trajectory length and rest of particles
i =0

Tmax = 720
fig, ax = plt.subplots()
data_arrays_tracer = list(data['tracer'][coriolis][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):
    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,marker,color=color2, zorder = i)
            legend.append('Re$_p^{\\mathrm{In}}$ ='+' {:d}'.format(Rep))
            i-=5
    else:
        print(pt)
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        print(np.max(pdf))
        ax.plot(bins,pdf,marker,color=color)
        legend.append(simtype[pt])

    # ax.set_title(simtype[pt])
    ax.legend(legend,fontsize=18,handlelength = 1.5)#,loc=(1,0.2))
    ax.set_ylabel("PDF")
    ax.set_xlabel("tidal av trajectory length difference to tracer [km]")
    ax.set_ylim(-0.005,0.255)
    ax.set_title('hourly velocity fields',fontsize=18)

    # ax.set_yscale('log')

In [ ]:
# not tidal av
# plot PDF difference trajectory length and rest of particles
i =0
Tmax=720
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))#,projection=ccrs.PlateCarree())
data_arrays_tracer = list(data['tracer'][coriolis][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):
    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length - combined_data_tracer.traj_length
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,marker,color=color2, zorder = i)
            legend.append('Re$_p^{\\mathrm{In}}$ ='+' {:d}'.format(Rep))
            i-=5
    else:
        print(pt)
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length - combined_data_tracer.traj_length
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        print(np.max(pdf))
        ax.plot(bins,pdf,marker,color=color)
        legend.append(simtype[pt])


ax.legend(legend,fontsize=18,handlelength = 1.5)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length difference to tracer [km]")
ax.set_title('hourly velocity fields',fontsize=18)
fig.savefig('../figures/hourly_vs_daily/PDF_trajectory_length_dif_hourly_field_Rep.pdf')

## Relative disspersion

In [ ]:
# Relative dispersion


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']
    ds_tracer = data['tracer'][coriolis][None][date]
    for pt in particle_types[1:]:
        for Rep in Repvalues[pt]:
            ds_Rep = data[pt][coriolis][Rep][date]
            dist= Haversine(ds_Rep['lon'],ds_Rep['lat'],
                                ds_tracer['lon'],ds_tracer['lat'])
            data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].assign(dist = dist)
            data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].rename({'dist':'dist_to_tracer'})

            #tial av
            dist_tidal_av= Haversine(ds_Rep['lon_tidal_av'],ds_Rep['lat_tidal_av'],
                                ds_tracer['lon_tidal_av'],ds_tracer['lat_tidal_av'])
            data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].assign(dist_tidal_av = dist_tidal_av)
            data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].rename({'dist_tidal_av':'dist_to_tracer_tidal_av'})
       

In [ ]:
# checking several month flexible Rep
pt = 'inertial_SM_drag_Rep'
Tmax=700
tlist=np.arange(0,Tmax,1)/24
fig,ax=plt.subplots()
legend=[]
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    ax.plot(tlist,data[pt][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[0:Tmax],'-')
    legend.append(date)
ax.legend(legend)
ax.set_xlabel('time [days]')
ax.set_ylabel('distance to tracer [km]')


In [ ]:
# plot distribution final distance between particles
tfinal = 799
markerlist=np.array(['--','-.',':','-','--','-.',':'])

i=0
Tmax=720
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[1:],markerlist):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('Re$_p^{\\mathrm{In}}=$'+ ' {:d}'.format(Rep))
    else:
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(simtype[pt])

ax.legend(legend,fontsize=18,handlelength=1.5)
ax.set_xlabel('t [days]')
ax.set_ylabel('relative distance to tracer [km]')
ax.set_title('hourly velocity fields',fontsize=18)
fig.savefig('../figures/hourly_vs_daily/relative_distance_hourly_Rep.pdf')


## distribution analysis
Now want to show that distribution itself is also a function of Reynolds number 
where with lower reynolds numbers there is more structure (divergence/convergence points)


In [ ]:
# show final shapshot
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'

Tmax=719

# ax.set_ylim(50,55)
for pt in particle_types[1:]:
    for Rep in Repvalues[pt]:
        fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
        sc =ax.scatter(data[pt][coriolis][Rep][date].lon[:,Tmax],data[pt][coriolis][Rep][date].lat[:,Tmax],c = data[pt][coriolis][Rep][date].dist_to_tracer[:,Tmax], marker='o',s=2,alpha=1,cmap = 'magma',vmin=0,vmax=50)
        fig.colorbar(sc,label='relative distance [km]',fraction=0.03,extend='max')
        ax.coastlines()
        title = simtype[pt]
        if (Rep != None):
            title += ' Re$_p^{\\mathrm{In}}$=' + f'{Rep:d}'
        ax.set_title(title)

        ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=0, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 20}
        gl.ylabel_style =  {'size': 20}


